```shell
nohup target/appassembler/bin/SearchCollection -topicreader Trec -index /tuna1/indexes/lucene-index.robust04.pos+docvectors+rawdocs/ -topics src/main/resources/topics-and-qrels/topics.robust04.301-450.601-700.txt -output run.robust04.bm25.topics.robust04.301-450.601-700.txt -bm25 & 
```

```shell
python gen_test_robust.py
```

In [19]:
pwd

'/home/w85yang/Anserini/Anserini'

In [10]:
%%time
# generate data from qrels file through SimpleSearcher
ftopic = "src/main/resources/topics-and-qrels/topics.robust04.301-450.601-700.txt"
fqrel = "src/main/resources/topics-and-qrels/qrels.robust2004.txt"
qid2query = get_qid2query(ftopic)
qid2reldocids = get_qid2reldocids(fqrel)


CPU times: user 352 ms, sys: 4 ms, total: 356 ms
Wall time: 353 ms


In [42]:
len(qid2query)

250

In [30]:
import string
printable = set(string.printable)
printable.remove("\n")
printable.remove("\t")
printable.remove("\r")
import json 

from nltk.tokenize import word_tokenize

In [9]:
def get_qid2query(ftopic):
    qid2query = {}
    f = open(ftopic)
    query_tag = "title"
    empty = False
    for l in f:
        if empty == True:
            qid2query[qid] = l.replace("\n", "").strip()
            empty = False
        ind = l.find("Number: ")
        if ind >= 0:
            qid = l[ind+8:-1]
            qid = int(qid) 
        ind = l.find("<{}>".format(query_tag))
        if ind >= 0:
            query = l[ind+8:-1].strip()
            if len(query) == 0:
                empty = True
            else:
                qid2query[qid] = query
    return qid2query

def getdoc(docid, folder):
    f = open("{}/{}".format(folder, docid))
    see_text = False
    doc = ""
    for l in f:
        l = l.replace("\n", "").strip()
        if l == "</TEXT>":
            break
        if see_text:
            if len(l) > 0:
                doc += l + " "
        elif "[Text]" in l:
            see_text = True
            doc += l[len("[Text]") + 1:] + " "
    return doc.strip()

def get_qid2reldocids(fqrel):
    f = open(fqrel)
    qid2reldocids = {}
    for l in f:
        qid, _, docid, score = l.replace("\n", "").strip().split()
        qid = int(qid)
        if score != "0":
            if qid not in qid2reldocids:
                qid2reldocids[qid] = set()
            qid2reldocids[qid].add(docid)
    return qid2reldocids

def get_doc_from_index(indexes, docid):
    doc_raw = indexes.getRawDocument(JString(docid))
    doc = json.loads(doc_raw)['text']
    return doc

In [40]:
docid = "FBIS3-10000"
folder = "qrels.robust04.docids.txt_rawdocs.dump"
getdoc(docid, folder)

'The Capois Lamort Organization [OCL] of Franck Pierre asks the parliamentarians to make a ruling, to choose one of three resolutions resulting from the congresses and conferences held in the country recently -- that from the Committee of Actions for National Reconciliation, that from the Group of Citizens of the Greater North, and that from the private sector -- in order to achieve a happy way out of the crisis. [Begin Pierre recording, in Creole] We in the OCL ask all conscientious and nationalistic parliamentarians to choose one of these three resolutions that can (?contribute) to the welfare of the Haitian people. We in the OCL ask that the Haitian Army -- in the event that the parliament cannot do its job of choosing one of the three resolutions -- go into action and form a military junta. That military junta should (?direct) the National Palace so it can manage the Haitian crisis and lead the country properly until elections are held. [end recording]'

In [11]:
def split_train_dev_test(qid2query):
    qid2query_train = {}
    qid2query_dev = {}
    qid2query_test = {}
    count = 0
    for qid in qid2query:
        count += 1
        if count % 10 == 0:
            qid2query_dev[qid] = qid2query[qid]
        elif count % 10 == 1:
            qid2query_test[qid] = qid2query[qid]
        else:
            qid2query_train[qid] = qid2query[qid]
    return qid2query_train, qid2query_dev, qid2query_test



In [12]:
qid2query_train, qid2query_dev, qid2query_test = split_train_dev_test(qid2query)

In [15]:
# f = open("run.robust04.bm25.topics.robust04.301-450.601-700.txt")
# f2 = open("run.robust04_test.bm25.topics.robust04.301-450.601-700.txt", "w")
f = open("run.simplesearcher.robust04.txt")
f2 = open("run.simplesearcher.robust04_test.txt", "w")
for l in f:
    qid = l.split()[0]
    qid = int(qid)
    if qid in qid2query_test:
        f2.write(l)
f2.close()

In [55]:
f1 = open("src/main/python/rerank/MatchZoo/data/robust04/Robust04Corpus/qrels.txt")
f2 = open("src/main/python/rerank/MatchZoo/data/robust04/Robust04Corpus/ql.txt")
ftrain = open("src/main/python/rerank/MatchZoo/data/robust04/Robust04Corpus/train_qrels.txt", "w")
fdev = open("src/main/python/rerank/MatchZoo/data/robust04/Robust04Corpus/dev_ql.txt", "w")
ftest = open("src/main/python/rerank/MatchZoo/data/robust04/Robust04Corpus/test_ql.txt", "w")
for l in f1:
    sim, a, b, qid, docid = l.replace("\n", "").split("\t")
    qid = int(qid)
    if qid in qid2query_train:
        ftrain.write(l)
for l in f2:
    sim, a, b, qid, docid = l.replace("\n", "").split("\t")
    qid = int(qid)
    if qid in qid2query_dev:
        fdev.write(l)
    if qid in qid2query_test:
        ftest.write(l)
        
ftrain.close()
fdev.close()
ftest.close()

In [1]:
qlen = []
for qid in qid2query:
    qlen.append(len(qid2query[qid].split()))
    
doclen = []
f2 = open("src/main/python/rerank/MatchZoo/data/robust04/Robust04Corpus/ql.txt")
for l in f2:
    sim, a, b, qid, docid = l.replace("\n", "").split("\t")
    doclen.append(len(b.split()))
    

In [28]:
def gen_qrel_data(f_input, f_output, doc_folder, remove_url=True, tokenize=True):
    count = 0
    with open(f_output, "w") as out, open(f_input) as f:
        for l in f:
            qid, _, docid, sim = l.replace("\n", "").split()
            qid = int(qid)
            if sim == "2":
                sim = "1"
            if sim == "-2":
                sim = "0"
            assert sim in ["0", "1"]
            a = qid2query[qid]
            try:
                b = getdoc(docid, doc_folder)
            except Exception as e:
                count += 1
                raise(e)
                continue
            if b is None:
                continue
            # remove characters that cannot be encoded by ASCII
            b = "".join(filter(lambda x: x in printable, b))
            if remove_url:
                bs = []
                for w in b.split():
                    if not w.startswith("http"):
                        bs.append(w)
                b = " ".join(bs)
            if tokenize:
                b = ' '.join(word_tokenize(b))
            out.write("{}\t{}\t{}\t{}\t{}\n".format(sim, a, b, qid, docid))
            out.flush()
    print("error docids: {}".format(count))
    

```shell
python run2docid.py # modify the path variable first
sh target/appassembler/bin/IndexUtils \
-index /tuna1/indexes/lucene-index.mb11.pos+docvectors+rawdocs \
-dumpRawDocs run.mb11.ql.topics.microblog2011.docids.txt
```

## Test and evaluation
```shell
cd src/main/python/rerank/MatchZoo/data/tweets
sh ./run_data.sh
cd ../../
python matchzoo/main.py --phase predict --model_file ./examples/tweets/config/duet_tweet.config
cd ../../../../../
./eval/trec_eval.9.0/trec_eval src/main/resources/topics-and-qrels/qrels.microblog2014.txt src/main/python/rerank/MatchZoo/data/tweets/predict.test.txt
```    

In [6]:
# try IndexUtil
year = 2014
f = open("src/main/resources/topics-and-qrels/qrels.robust2004.txt")
with open("qrels.robust04.docids.txt", "w") as out:
    for l in f:
        qid, _, docid, sim = l.replace("\n", "").split()
        out.write("{}\n".format(docid))

```shell
sh target/appassembler/bin/IndexUtils \
-index /tuna1/indexes/lucene-index.robust04.pos+docvectors+rawdocs \
-dumpRawDocs qrels.robust04.docids.txt
```

In [41]:
%%time
# generate data from qrels file\
qid2query = get_qid2query(ftopic)
qid2reldocids = get_qid2reldocids(fqrel)
f_output = "src/main/python/rerank/MatchZoo/data/robust04/Robust04Corpus/qrels.txt"
doc_folder = "qrels.robust04.docids.txt_rawdocs.dump"
gen_qrel_data(fqrel, f_output, doc_folder, tokenize=False, remove_url=False)


error docids: 0
CPU times: user 8min 8s, sys: 25 s, total: 8min 33s
Wall time: 16min 54s


```shell
cd src/main/python/rerank/MatchZoo/data/tweets/TweetCorpus/
cat train_qrels_2011.txt train_qrels_2012.txt train_qrels_2013.txt > train_qrels_2011_2012_2013_all.txt
python split_train_dev.py
```

## Train
```shell
cd src/main/python/rerank/MatchZoo/data/tweets
sh ./run_data.sh
cd ../../
python matchzoo/main.py --phase train --model_file ./examples/tweets/config/duet_tweet.config
```


In [20]:
lens = []
for qid in qid2query:
    lens.append(len(qid2query[qid].split()))